<a href="https://colab.research.google.com/github/Rohan5076/practice-code/blob/main/data_drift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
!pip install alibi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [51]:
!pip install alibi_detect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [52]:
import alibi
import matplotlib.pyplot as plt
import numpy as np

from alibi_detect.cd import ChiSquareDrift, TabularDrift
from alibi_detect.saving import save_detector, load_detector

In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from alibi_detect.od import IForest
from alibi_detect.datasets import fetch_kdd
from alibi_detect.utils.data import create_outlier_batch
from alibi_detect.utils.fetching import fetch_detector
from alibi_detect.utils.saving import save_detector, load_detector
from alibi_detect.utils.visualize import plot_instance_score, plot_roc

In [54]:
#load the data and First look for data
df = pd.read_csv('/content/drive/MyDrive/All CSV files-20220711T101939Z-001/All CSV files/Loan_Default.csv')
df

,ID,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,...,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Region,Security_Type,Status,dtir1
0,24890,2019,cf,Sex Not Available,nopre,type1,p1,l1,nopc,nob/c,...,EXP,758,CIB,25-34,to_inst,98.728814,south,direct,1,45.0
1,24891,2019,cf,Male,nopre,type2,p1,l1,nopc,b/c,...,EQUI,552,EXP,55-64,to_inst,NaN,North,direct,1,NaN
2,24892,2019,cf,Male,pre,type1,p1,l1,nopc,nob/c,...,EXP,834,CIB,35-44,to_inst,80.019685,south,direct,0,46.0
3,24893,2019,cf,Male,nopre,type1,p4,l1,nopc,nob/c,...,EXP,587,CIB,45-54,not_inst,69.376900,North,direct,0,42.0
4,24894,2019,cf,Joint,pre,type1,p1,l1,nopc,nob/c,...,CRIF,602,EXP,25-34,not_inst,91.886544,North,direct,0,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148665,173555,2019,cf,Sex Not Available,nopre,type1,p3,l1,nopc,nob/c,...,CIB,659,EXP,55-64,to_inst,71.792763,south,direct,0,48.0
148666,173556,2019,cf,Male,nopre,type1,p1,l1,nopc,nob/c,...,CIB,569,CIB,25-34,not_inst,74.428934,south,direct,0,15.0
148667,173557,2019,cf,Male,nopre,type1,p4,l1,nopc,nob/c,...,CIB,702,EXP,45-54,not_inst,61.332418,North,direct,0,49.0
148668,173558,2019,cf,Female,nopre,type1,p4,l1,nopc,nob/c,...,EXP,737,EXP,55-64,to_inst,70.683453,North,direct,0,29.0


In [58]:
df1=df
df2=df1

In [59]:
df["Status"].value_counts()

0    112031
1     36639
Name: Status, dtype: int64

In [60]:
over = RandomOverSampler(sampling_strategy=0.8)
df = pd.concat(over.fit_resample(df[df.columns[:-2]], df["Status"]), axis=1)
under = RandomUnderSampler(sampling_strategy=0.9)
df = pd.concat(under.fit_resample(df[df.columns[:-2]], df["Status"]), axis=1)

In [61]:
df["Status"].value_counts()

0    99582
1    89624
Name: Status, dtype: int64

In [62]:
df.head()

,ID,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,...,total_units,income,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Region,Status
0,50283,2019,cf,Joint,nopre,type1,p1,l1,nopc,nob/c,...,1U,10320.0,CIB,639,EXP,45-54,not_inst,80.405405,North,0
1,46805,2019,cf,Joint,nopre,type1,p1,l1,nopc,nob/c,...,1U,5340.0,EXP,592,EXP,35-44,not_inst,85.630841,south,0
2,140941,2019,cf,Sex Not Available,nopre,type1,p1,l1,nopc,nob/c,...,1U,3420.0,EXP,868,CIB,35-44,to_inst,95.542636,south,0
3,155250,2019,cf,Joint,nopre,type1,p4,l1,nopc,nob/c,...,1U,12180.0,EXP,537,EXP,55-64,not_inst,88.043478,south,0
4,110305,2019,cf,Male,nopre,type2,p3,l1,nopc,b/c,...,1U,2040.0,CRIF,712,CIB,35-44,to_inst,77.173913,North,0


In [63]:
def missing_values(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

In [64]:
# Missing values statistics
miss_values = missing_values(df)
miss_values.head(10)

Dataframe has 32 columns.
There are 13 columns that have missing values.


,Missing Values,% of Total Values
Upfront_charges,92036,48.6
Interest_rate_spread,89624,47.4
rate_of_interest,89117,47.1
property_value,36726,19.4
LTV,36726,19.4
income,10056,5.3
loan_limit,4334,2.3
approv_in_adv,1212,0.6
age,507,0.3
submission_of_application,507,0.3


In [65]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer()
df[['Interest_rate_spread','Upfront_charges','rate_of_interest', 'term','property_value','income','LTV']] = imputer.fit_transform(df[['Interest_rate_spread','Upfront_charges','rate_of_interest', 'term','property_value','income','LTV']])


imputer = SimpleImputer(strategy='most_frequent')
df[['age' ,'loan_limit']] = imputer.fit_transform(df[['age' , 'loan_limit']])

In [66]:
df.dropna(inplace = True)

In [69]:
df.isna().sum()

ID                           0
year                         0
loan_limit                   0
Gender                       0
approv_in_adv                0
loan_type                    0
loan_purpose                 0
Credit_Worthiness            0
open_credit                  0
business_or_commercial       0
loan_amount                  0
rate_of_interest             0
Interest_rate_spread         0
Upfront_charges              0
term                         0
Neg_ammortization            0
interest_only                0
lump_sum_payment             0
property_value               0
construction_type            0
occupancy_type               0
Secured_by                   0
total_units                  0
income                       0
credit_type                  0
Credit_Score                 0
co-applicant_credit_type     0
age                          0
submission_of_application    0
LTV                          0
Region                       0
Status                       0
dtype: i

In [70]:
objList = df.select_dtypes(include = "object").columns
print (objList)

Index(['loan_limit', 'Gender', 'approv_in_adv', 'loan_type', 'loan_purpose',
       'Credit_Worthiness', 'open_credit', 'business_or_commercial',
       'Neg_ammortization', 'interest_only', 'lump_sum_payment',
       'construction_type', 'occupancy_type', 'Secured_by', 'total_units',
       'credit_type', 'co-applicant_credit_type', 'age',
       'submission_of_application', 'Region'],
      dtype='object')


In [71]:
#Label Encoding for object to numeric conversion
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for feat in objList:
    df[feat] = le.fit_transform(df[feat].astype(str))

print (df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187176 entries, 0 to 189205
Data columns (total 32 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ID                         187176 non-null  int64  
 1   year                       187176 non-null  int64  
 2   loan_limit                 187176 non-null  int64  
 3   Gender                     187176 non-null  int64  
 4   approv_in_adv              187176 non-null  int64  
 5   loan_type                  187176 non-null  int64  
 6   loan_purpose               187176 non-null  int64  
 7   Credit_Worthiness          187176 non-null  int64  
 8   open_credit                187176 non-null  int64  
 9   business_or_commercial     187176 non-null  int64  
 10  loan_amount                187176 non-null  int64  
 11  rate_of_interest           187176 non-null  float64
 12  Interest_rate_spread       187176 non-null  float64
 13  Upfront_charges            18

In [72]:
df

,ID,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,...,total_units,income,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Region,Status
0,50283,2019,0,1,0,0,0,0,0,1,...,0,10320.000000,0,639,1,2,0,80.405405,0,0
1,46805,2019,0,1,0,0,0,0,0,1,...,0,5340.000000,3,592,1,1,0,85.630841,3,0
2,140941,2019,0,3,0,0,0,0,0,1,...,0,3420.000000,3,868,0,1,1,95.542636,3,0
3,155250,2019,0,1,0,0,3,0,0,1,...,0,12180.000000,3,537,1,3,0,88.043478,3,0
4,110305,2019,0,2,0,1,2,0,0,0,...,0,2040.000000,1,712,0,1,1,77.173913,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189201,33694,2019,0,0,0,0,0,0,0,1,...,0,2100.000000,3,730,0,0,1,51.689189,0,1
189202,46990,2019,0,1,0,2,3,1,0,1,...,0,6731.966509,3,767,1,6,1,149.637681,0,1
189203,72163,2019,0,0,1,0,2,0,0,1,...,0,9360.000000,2,893,1,3,0,73.550021,3,1
189204,44937,2019,1,2,0,0,0,0,0,1,...,0,7860.000000,2,591,1,0,0,73.550021,0,1


In [73]:
X=df.drop("Status",axis=1)

In [74]:
X

,ID,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,...,Secured_by,total_units,income,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Region
0,50283,2019,0,1,0,0,0,0,0,1,...,0,0,10320.000000,0,639,1,2,0,80.405405,0
1,46805,2019,0,1,0,0,0,0,0,1,...,0,0,5340.000000,3,592,1,1,0,85.630841,3
2,140941,2019,0,3,0,0,0,0,0,1,...,0,0,3420.000000,3,868,0,1,1,95.542636,3
3,155250,2019,0,1,0,0,3,0,0,1,...,0,0,12180.000000,3,537,1,3,0,88.043478,3
4,110305,2019,0,2,0,1,2,0,0,0,...,0,0,2040.000000,1,712,0,1,1,77.173913,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189201,33694,2019,0,0,0,0,0,0,0,1,...,0,0,2100.000000,3,730,0,0,1,51.689189,0
189202,46990,2019,0,1,0,2,3,1,0,1,...,0,0,6731.966509,3,767,1,6,1,149.637681,0
189203,72163,2019,0,0,1,0,2,0,0,1,...,0,0,9360.000000,2,893,1,3,0,73.550021,3
189204,44937,2019,1,2,0,0,0,0,0,1,...,0,0,7860.000000,2,591,1,0,0,73.550021,0


In [75]:
y=df["Status"]

In [76]:
X=X.to_numpy()

In [77]:
y=y.to_numpy()

In [78]:
X

array([[5.02830000e+04, 2.01900000e+03, 0.00000000e+00, ...,
        0.00000000e+00, 8.04054054e+01, 0.00000000e+00],
       [4.68050000e+04, 2.01900000e+03, 0.00000000e+00, ...,
        0.00000000e+00, 8.56308411e+01, 3.00000000e+00],
       [1.40941000e+05, 2.01900000e+03, 0.00000000e+00, ...,
        1.00000000e+00, 9.55426357e+01, 3.00000000e+00],
       ...,
       [7.21630000e+04, 2.01900000e+03, 0.00000000e+00, ...,
        0.00000000e+00, 7.35500208e+01, 3.00000000e+00],
       [4.49370000e+04, 2.01900000e+03, 1.00000000e+00, ...,
        0.00000000e+00, 7.35500208e+01, 0.00000000e+00],
       [1.63057000e+05, 2.01900000e+03, 0.00000000e+00, ...,
        1.00000000e+00, 7.35500208e+01, 3.00000000e+00]])

In [79]:
df.shape

(187176, 32)

In [80]:
df.columns

Index(['ID', 'year', 'loan_limit', 'Gender', 'approv_in_adv', 'loan_type',
       'loan_purpose', 'Credit_Worthiness', 'open_credit',
       'business_or_commercial', 'loan_amount', 'rate_of_interest',
       'Interest_rate_spread', 'Upfront_charges', 'term', 'Neg_ammortization',
       'interest_only', 'lump_sum_payment', 'property_value',
       'construction_type', 'occupancy_type', 'Secured_by', 'total_units',
       'income', 'credit_type', 'Credit_Score', 'co-applicant_credit_type',
       'age', 'submission_of_application', 'LTV', 'Region', 'Status'],
      dtype='object')

In [81]:
feature_names=['ID', 'year', 'loan_limit', 'Gender', 'approv_in_adv', 'loan_type',
       'loan_purpose', 'Credit_Worthiness', 'open_credit',
       'business_or_commercial', 'loan_amount', 'rate_of_interest',
       'Interest_rate_spread', 'Upfront_charges', 'term', 'Neg_ammortization',
       'interest_only', 'lump_sum_payment', 'property_value',
       'construction_type', 'occupancy_type', 'Secured_by', 'total_units',
       'income', 'credit_type', 'Credit_Score', 'co-applicant_credit_type',
       'age', 'submission_of_application', 'LTV', 'Region']

In [82]:
n_ref = 60000
n_test = 60000

X_ref, X_t0, X_t1 = X[:n_ref], X[n_ref:n_ref + n_test], X[n_ref + n_test:n_ref + 2 * n_test]
X_ref.shape, X_t0.shape, X_t1.shape

((60000, 31), (60000, 31), (60000, 31))

In [87]:
y_ref, y_t0, y_t1 = y[:n_ref], y[n_ref:n_ref + n_test], y[n_ref + n_test:n_ref + 2 * n_test]
y_ref.shape, y_t0.shape, y_t1.shape

((60000,), (60000,), (60000,))

In [83]:
X.shape, y.shape

((187176, 31), (187176,))

In [84]:
cd = TabularDrift(X_ref, p_val=.05)

In [88]:
cd1 = TabularDrift(y_ref, p_val=.05)

In [34]:
filepath = 'my_path'  # change to directory where detector is saved
save_detector(cd, filepath)
cd = load_detector(filepath)

In [35]:
preds = cd.predict(X_t0)
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds['data']['is_drift']]))

Drift? Yes!


In [37]:
categories_per_feature=0

In [42]:
for f in range(cd.n_features):
    stat = 'Chi2' if f in list() else 'K-S'
    fname = feature_names[f]
    stat_val, p_val = preds['data']['distance'][f], preds['data']['p_val'][f]
    print(f'{fname} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

ID -- K-S 0.149 -- p-value 0.000
year -- K-S 0.000 -- p-value 1.000
loan_limit -- K-S 0.012 -- p-value 0.001
Gender -- K-S 0.024 -- p-value 0.000
approv_in_adv -- K-S 0.012 -- p-value 0.001
loan_type -- K-S 0.027 -- p-value 0.000
loan_purpose -- K-S 0.010 -- p-value 0.006
Credit_Worthiness -- K-S 0.005 -- p-value 0.334
open_credit -- K-S 0.000 -- p-value 1.000
business_or_commercial -- K-S 0.023 -- p-value 0.000
loan_amount -- K-S 0.026 -- p-value 0.000
rate_of_interest -- K-S 0.203 -- p-value 0.000
Interest_rate_spread -- K-S 0.192 -- p-value 0.000
Upfront_charges -- K-S 0.203 -- p-value 0.000
term -- K-S 0.009 -- p-value 0.024
Neg_ammortization -- K-S 0.042 -- p-value 0.000
interest_only -- K-S 0.004 -- p-value 0.698
lump_sum_payment -- K-S 0.022 -- p-value 0.000
property_value -- K-S 0.080 -- p-value 0.000
construction_type -- K-S 0.000 -- p-value 1.000
occupancy_type -- K-S 0.006 -- p-value 0.211
Secured_by -- K-S 0.000 -- p-value 1.000
total_units -- K-S 0.003 -- p-value 0.888
inc

In [43]:
preds['data']['threshold']

0.0016129032258064516

In [44]:
fpreds = cd.predict(X_t0, drift_type='feature')

In [45]:
for f in range(cd.n_features):
    stat = 'Chi2' if f in list() else 'K-S'
    fname = feature_names[f]
    is_drift = fpreds['data']['is_drift'][f]
    stat_val, p_val = fpreds['data']['distance'][f], fpreds['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

ID -- Drift? Yes! -- K-S 0.149 -- p-value 0.000
year -- Drift? No! -- K-S 0.000 -- p-value 1.000
loan_limit -- Drift? Yes! -- K-S 0.012 -- p-value 0.001
Gender -- Drift? Yes! -- K-S 0.024 -- p-value 0.000
approv_in_adv -- Drift? Yes! -- K-S 0.012 -- p-value 0.001
loan_type -- Drift? Yes! -- K-S 0.027 -- p-value 0.000
loan_purpose -- Drift? Yes! -- K-S 0.010 -- p-value 0.006
Credit_Worthiness -- Drift? No! -- K-S 0.005 -- p-value 0.334
open_credit -- Drift? No! -- K-S 0.000 -- p-value 1.000
business_or_commercial -- Drift? Yes! -- K-S 0.023 -- p-value 0.000
loan_amount -- Drift? Yes! -- K-S 0.026 -- p-value 0.000
rate_of_interest -- Drift? Yes! -- K-S 0.203 -- p-value 0.000
Interest_rate_spread -- Drift? Yes! -- K-S 0.192 -- p-value 0.000
Upfront_charges -- Drift? Yes! -- K-S 0.203 -- p-value 0.000
term -- Drift? Yes! -- K-S 0.009 -- p-value 0.024
Neg_ammortization -- Drift? Yes! -- K-S 0.042 -- p-value 0.000
interest_only -- Drift? No! -- K-S 0.004 -- p-value 0.698
lump_sum_payment -- 

In [46]:
preds = cd.predict(X_t1)
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds['data']['is_drift']]))

Drift? Yes!


In [47]:
for f in range(cd.n_features):
    stat = 'Chi2' if f in list(()) else 'K-S'
    fname = feature_names[f]
    is_drift = (preds['data']['p_val'][f] < preds['data']['threshold']).astype(int)
    stat_val, p_val = preds['data']['distance'][f], preds['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

ID -- Drift? Yes! -- K-S 0.144 -- p-value 0.000
year -- Drift? No! -- K-S 0.000 -- p-value 1.000
loan_limit -- Drift? Yes! -- K-S 0.028 -- p-value 0.000
Gender -- Drift? Yes! -- K-S 0.075 -- p-value 0.000
approv_in_adv -- Drift? Yes! -- K-S 0.031 -- p-value 0.000
loan_type -- Drift? Yes! -- K-S 0.073 -- p-value 0.000
loan_purpose -- Drift? Yes! -- K-S 0.034 -- p-value 0.000
Credit_Worthiness -- Drift? Yes! -- K-S 0.016 -- p-value 0.000
open_credit -- Drift? No! -- K-S 0.001 -- p-value 1.000
business_or_commercial -- Drift? Yes! -- K-S 0.072 -- p-value 0.000
loan_amount -- Drift? Yes! -- K-S 0.074 -- p-value 0.000
rate_of_interest -- Drift? Yes! -- K-S 0.581 -- p-value 0.000
Interest_rate_spread -- Drift? Yes! -- K-S 0.543 -- p-value 0.000
Upfront_charges -- Drift? Yes! -- K-S 0.566 -- p-value 0.000
term -- Drift? Yes! -- K-S 0.021 -- p-value 0.000
Neg_ammortization -- Drift? Yes! -- K-S 0.108 -- p-value 0.000
interest_only -- Drift? No! -- K-S 0.011 -- p-value 0.002
lump_sum_payment --

In [92]:
def plot_categories(idx: int) -> None:
    # reference data
    x_ref_count = {f: [(X_ref[:, f] == v).sum() for v in vals]
                   for f, vals in cd.x_ref_categories.items()}
    fref_drift = {cat: x_ref_count[idx][i] for i, cat in enumerate(category_map[idx])}

    # test set
    cats = {f: list(np.unique(X_t1[:, f])) for f in categories_per_feature.keys()}
    X_count = {f: [(X_t1[:, f] == v).sum() for v in vals] for f, vals in cats.items()}
    fxt1_drift = {cat: X_count[idx][i] for i, cat in enumerate(category_map[idx])}

    # plot bar chart
    plot_labels = list(fxt1_drift.keys())
    ind = np.arange(len(plot_labels))
    width = .35
    fig, ax = plt.subplots()
    p1 = ax.bar(ind, list(fref_drift.values()), width)
    p2 = ax.bar(ind + width, list(fxt1_drift.values()), width)
    ax.set_title(f'Counts per category for {feature_names[idx]} feature')
    ax.set_xticks(ind + width / 2)
    ax.set_xticklabels(plot_labels)
    ax.legend((p1[0], p2[0]), ('Reference', 'Test'), loc='upper right', ncol=2)
    ax.set_ylabel('Counts')
    ax.set_xlabel('Categories')
    plt.xticks(list(np.arange(len(plot_labels))), plot_labels, rotation='vertical')
    plt.show()